## Olympics Prediction Model

In [ ]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 fosforio fosforml numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install tensorflow[and-cuda]

In [ ]:
import os
os._exit(00)

In [ ]:
# Import packages

In [1]:
from fosforio import snowflake
from fosforml import *
from fosforml.constants import MLModelFlavours
from fosforio import get_dataframe
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
import warnings; warnings.simplefilter('ignore')
from joblib import dump, load
import requests
from tqdm import tqdm
import time
import calendar

from time import sleep
import configparser
from dateutil.relativedelta import relativedelta
import datetime
from dateutil.easter import easter
from scipy.optimize import minimize_scalar
from scipy.optimize import curve_fit

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


/tmp/pip_packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Matplotlib created a temporary cache directory at /tmp/matplotlib-io0mi4oj because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
#Import Data

snowflake.get_connection(connection_name="ME_OLYMPICS_CNX")
data = get_dataframe("ATHLETE_EVENTS")
data

Exception occurred in getting snowflake connection: 'connectionSources'
Reading dataframe from snowflake native connector


,ID,NAME,SEX,AGE,HEIGHT,WEIGHT,TEAM,NOC,GAMES,YEAR,SEASON,CITY,SPORT,EVENT,MEDAL
0,1,A Dijiang,M,24,180,80,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NA
1,2,A Lamusi,M,23,170,60,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NA
2,3,Gunnar Nielsen Aaby,M,24,NA,NA,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NA
3,4,Edgar Lindenau Aabye,M,34,NA,NA,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21,185,82,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29,179,89,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NA
271112,135570,Piotr ya,M,27,176,59,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NA
271113,135570,Piotr ya,M,27,176,59,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NA
271114,135571,Tomasz Ireneusz ya,M,30,185,96,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NA


In [4]:
data_bkp = get_dataframe("ATHLETE_EVENTS")
data_bkp

Reading dataframe from snowflake native connector


,ID,NAME,SEX,AGE,HEIGHT,WEIGHT,TEAM,NOC,GAMES,YEAR,SEASON,CITY,SPORT,EVENT,MEDAL
0,1,A Dijiang,M,24,180,80,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NA
1,2,A Lamusi,M,23,170,60,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NA
2,3,Gunnar Nielsen Aaby,M,24,NA,NA,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NA
3,4,Edgar Lindenau Aabye,M,34,NA,NA,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21,185,82,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29,179,89,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NA
271112,135570,Piotr ya,M,27,176,59,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NA
271113,135570,Piotr ya,M,27,176,59,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NA
271114,135571,Tomasz Ireneusz ya,M,30,185,96,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NA


#EDA
data['AGE'].value_counts().to_dict()
data.info()
data['AGE'][data['AGE']!='NA'].astype(int).median()
data['WEIGHT'][data['WEIGHT']!='NA'].astype(float).median()
data['HEIGHT'][data['HEIGHT']!='NA'].astype(int).median()

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   ID      271116 non-null  int32 
 1   NAME    271116 non-null  object
 2   SEX     271116 non-null  object
 3   AGE     271116 non-null  object
 4   HEIGHT  271116 non-null  object
 5   WEIGHT  271116 non-null  object
 6   TEAM    271116 non-null  object
 7   NOC     271116 non-null  object
 8   GAMES   271116 non-null  object
 9   YEAR    271116 non-null  int16 
 10  SEASON  271116 non-null  object
 11  CITY    271116 non-null  object
 12  SPORT   271116 non-null  object
 13  EVENT   271116 non-null  object
 14  MEDAL   271116 non-null  object
dtypes: int16(1), int32(1), object(13)
memory usage: 28.4+ MB


In [6]:
data.columns = data.columns.str.lower()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      271116 non-null  int32 
 1   name    271116 non-null  object
 2   sex     271116 non-null  object
 3   age     271116 non-null  object
 4   height  271116 non-null  object
 5   weight  271116 non-null  object
 6   team    271116 non-null  object
 7   noc     271116 non-null  object
 8   games   271116 non-null  object
 9   year    271116 non-null  int16 
 10  season  271116 non-null  object
 11  city    271116 non-null  object
 12  sport   271116 non-null  object
 13  event   271116 non-null  object
 14  medal   271116 non-null  object
dtypes: int16(1), int32(1), object(13)
memory usage: 28.4+ MB


In [7]:
data = data.drop(['id', 'name', 'games','event', 'noc'], axis=1)

data['WEIGHT'] = data['WEIGHT'].replace('NA',str(float(data['WEIGHT'][data['WEIGHT']!='NA'].astype(float).median()))).astype(float)

data['WEIGHT'].value_counts().to_dict()

data.info()

In [9]:
data['age'] = data['age'].replace('NA',str(int(data['age'][data['age']!='NA'].astype(int).median()))).astype(int)

In [10]:
data['height'] = data['height'].replace('NA',str(int(data['height'][data['height']!='NA'].astype(int).median()))).astype(int)

In [11]:
tf.random.set_seed(100)

# CREATING LABELS

In [12]:
data['medal'].value_counts()

medal
NA        231333
Gold       13372
Bronze     13295
Silver     13116
Name: count, dtype: int64

In [16]:
data['medals']= data['medal'].map({'Gold': 1, 'Silver': 2 , 'Bronze': 3, 'NA' : 0}) 

data['medals'].value_counts()
data['medal'] = data['medal'].apply(lambda x: 1 if str(x) != 'nan' else 0)

In [20]:
# taking a copy of data at this stage
data_bkp_step2 = data

In [23]:
#Dropping columns as necessary,
#weight correlated with height, hence dropping

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   sex     271116 non-null  object
 1   age     271116 non-null  int64 
 2   height  271116 non-null  int64 
 3   weight  271116 non-null  object
 4   team    271116 non-null  object
 5   year    271116 non-null  int16 
 6   season  271116 non-null  object
 7   city    271116 non-null  object
 8   sport   271116 non-null  object
 9   medal   271116 non-null  object
 10  medals  271116 non-null  int64 
dtypes: int16(1), int64(3), object(7)
memory usage: 21.2+ MB


In [ ]:
data = data.drop(['weight], axis=1)

In [ ]:
data['age'].value_counts()

# Cleaning data

In [22]:
# converting datatypes
data = data.astype({'age': int, 'height': int, 'weight' : int})
print(data.dtypes)

ValueError: invalid literal for int() with base 10: 'NA': Error while type casting for column 'weight'

In [ ]:
data = data.drop(['id', 'name', 'games'], axis=1)

In [ ]:
data.isna().mean()

In [ ]:
print("Total missing values:", data.isna().sum().sum())

In [ ]:
data['year'].value_counts().sort_values()

In [ ]:
data['season'].value_counts()

In [ ]:
filtered_data = data[data['season'] == 'Summer'].reset_index()

In [ ]:
filtered_data['season'].value_counts()

In [ ]:
f_data = filtered_data[filtered_data['year'] > 1960].reset_index()
f_data.info()

In [ ]:
f_data['year'].value_counts()

In [ ]:
f_data

In [ ]:
f_data = f_data.drop(['event'], axis=1)

In [ ]:
f_data = f_data.drop(['noc', 'level_0', 'index'], axis=1)

In [ ]:
f_data1 = f_data[f_data['sport'].map(f_data['sport'].value_counts()) > 1500]

In [ ]:
f_data1['sport'].value_counts()

In [ ]:
f_data2 = f_data1[f_data1['team'].map(f_data1['team'].value_counts()) > 1500]

In [ ]:
f_data2['team'].value_counts()

In [ ]:
f_data2['year'].value_counts()

# Over all filters:
Season: Summer only
Year: >1960 only
SPorts that had more than 1500 values
Team: that had more than 1500 values
    removed rep. columns like noc, event, etc.
    removing weight column as it is highly correlated to height column

# Encoding Data

In [ ]:
f_data2['medal'].value_counts()

In [ ]:
f_data2 = f_data2.drop(['height'], axis=1)

In [ ]:
{column: len(f_data2[column].unique()) for column in f_data2.select_dtypes('object').columns}

In [ ]:
def binary_encode(df, columns, positive_values):
    df = df.copy()
    for column, positive_value in zip(columns, positive_values):
        df[column] = df[column].apply(lambda x: 1 if x == positive_value else 0)
    return df

def onehot_encode(df, columns, prefixes):
    df = df.copy()
    for column, prefix in zip(columns, prefixes):
        dummies = pd.get_dummies(df[column], prefix=prefix)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(column, axis=1)
    return df

#df = binary_encode(
    f_data2,
    columns=['sex', 'season'],
    positive_values=['M', 'Summer']
)



In [ ]:
df1 = onehot_encode(
    f_data2,
    columns=['sex','team', 'city', 'sport'],
    prefixes=['sex','t', 'c', 's']
)

In [ ]:
print("Remaining non-numeric columns:", len(df1.select_dtypes('object').columns))

In [ ]:
df1.info()

In [ ]:
df1 = df1.drop(['season'], axis=1)

In [ ]:
df1.reset_index()
df1.info()

# Visualizing Correlations

In [ ]:
corr = f_data2.loc[:, :'medal'].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=True, vmin=-1.0, cmap='mako')
plt.show()

In [ ]:
df1.info()

In [ ]:
grouped_data = df1.groupby('team')['medal'].sum()

# Splitting/Scaling

In [ ]:
#data

In [ ]:
y = df1['medal'].copy()
X = df1.drop('medal', axis=1).copy()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=100)

# Training

In [ ]:
X.shape

In [ ]:
print("Class Distribution (Positive to Negative): {:.1f}% / {:.1f}%".format(y_train.mean() * 100, (1 - y_train.mean()) * 100))

In [ ]:
inputs = tf.keras.Input(shape=(X.shape[1]))
x = tf.keras.layers.Dense(64, activation='relu')(inputs)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs)


model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc')
    ]
)


history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

In [ ]:
y_train

# Results

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_true = np.array(y_test)
y_pred = np.squeeze(np.array(model.predict(X_test) >= 0.5, dtype=int))

In [ ]:
print("Classification Report:\n\n", classification_report(y_true, y_pred))

In [ ]:
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

In [ ]:
y_pred